In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

# Step 1: Import train.csv and test.csv
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Store the original test ID for reference later
test_ids = test.index

# Step 2: Concatenate the train and test datasets
# Create a new column 'Source' to distinguish between train and test data
train['Source'] = 1
test['Source'] = 0

# Concatenate train and test data
combined = pd.concat([train, test], axis=0, ignore_index=True)

# Step 3: Check for Null Values and analyze it in combined data
null_values = combined.isnull().sum()
print("Null Values in each column:\n", null_values[null_values > 0])
print("\nTotal Null Values: ", null_values.sum())

# Separate numeric and categorical columns
numeric_cols = combined.select_dtypes(include=['number']).columns
categorical_cols = combined.select_dtypes(include=['object']).columns

# Fill missing values for numeric columns with the median
combined[numeric_cols] = combined[numeric_cols].fillna(combined[numeric_cols].median())

# Fill missing values for categorical columns with the mode (most frequent value)
for col in categorical_cols:
    mode_val = combined[col].mode()[0]  # Get the most frequent value
    combined[col].fillna(mode_val, inplace=True)

# Drop features with more than 500 null values
null_values = combined.isnull().sum()
features_to_drop = null_values[null_values > 500].index
print(f"\nDropping features with more than 500 null values: {features_to_drop.tolist()}")
combined.drop(columns=features_to_drop, inplace=True)

# Update categorical columns after dropping features
categorical_cols = combined.select_dtypes(include=['object']).columns

# Step 4: One-hot encoding for each string-based feature
# Apply one-hot encoding to categorical variables
combined = pd.get_dummies(combined, columns=categorical_cols, drop_first=True)

print("Data after one-hot encoding:\n", combined.head())

# Step 5: Split combined data back into train and test sets
train_data = combined[combined['Source'] == 1].drop('Source', axis=1)
test_data = combined[combined['Source'] == 0].drop('Source', axis=1)

# Ensure the columns match exactly between train and test data
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

# Step 6: Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Define base models and stacking model
estimators = [
    ('xgb', XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=10, random_state=42)),
    ('lgbm', lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=31, random_state=42)),
    ('catboost', CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=10, random_seed=42, verbose=0))
]

# Initialize the stacking model with Ridge as the final estimator
stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge())

# Train the stacking model on the training data
stacking_model.fit(X_train, y_train)

# Step 8: Predict on the validation set
y_val_pred = stacking_model.predict(X_val)

# Calculate the logarithm of the predictions and the actual values
y_val_log_pred = np.log1p(y_val_pred)
y_val_log_actual = np.log1p(y_val)

# Step 9: Calculate Logarithmic RMSE and R² score
log_rmse = np.sqrt(mean_squared_error(y_val_log_actual, y_val_log_pred))
r2 = r2_score(y_val_log_actual, y_val_log_pred)

print(f"\nValidation Logarithmic RMSE: {log_rmse}")
print(f"Validation R² Score: {r2}")

# Step 10: Predict on the test set using the entire training data
stacking_model.fit(X, y)  # Re-train on the entire dataset before predicting on the test set
y_test_pred = stacking_model.predict(test_data[X_train.columns])

# Handle any extremely large or small predictions
y_test_pred = np.clip(y_test_pred, 0, np.percentile(y_test_pred, 99))

# Step 11: Save the predictions to a CSV file
output = pd.DataFrame({'Id': test_ids, 'SalePrice': y_test_pred})
output.to_csv('test_predictions.csv', index=False)

print("\nPredictions saved to test_predictions.csv")


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Null Values in each column:
 MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType      1766
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

Total Null Values:  17166

Dropping features with more than 500 null values: []
Data after one-hot encoding:
    Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0  